In [1]:
import pandas as pd
import numpy as np

# Load ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

# Transformers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Others
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,plot_confusion_matrix

from sklearn.multioutput import MultiOutputClassifier

In [2]:
raw_data = pd.read_csv('OUT.csv')
raw_data.shape
raw_data.head()

,Name,Pattern Type,Neckline
0,Lettuce Trim Rib-knit Tie Dye Top,Tie Dye,Scoop Neck
1,Slant Pocket Solid Skirt,Plain,NaN
2,Contrast Binding Tie Dye Shorts,Tie Dye,NaN
3,Square Neck Plaid Dress,Plaid,Square Neck
4,Pinstriped Mid-Rise Dolphin Shorts,Striped,NaN


In [3]:
import nltk
import unicodedata
from nltk.corpus import stopwords
import re

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = re.sub(r"text+","",w) #removing "text" from every sentence
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

#CLEANING SENTENCES
sentences = raw_data['Name']
sentences = sentences.astype(str)
sentences = sentences.map(preprocess_sentence)
raw_data['Name'] = sentences
raw_data.head()

,Name,Pattern Type,Neckline
0,lettuce trim rib knit tie dye top,Tie Dye,Scoop Neck
1,slant pocket solid skirt,Plain,NaN
2,contrast binding tie dye shorts,Tie Dye,NaN
3,square neck plaid dress,Plaid,Square Neck
4,pinstriped mid rise dolphin shorts,Striped,NaN


In [4]:
data = raw_data

In [5]:
# Features & Labels

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna("NA", inplace=True)

Xfeatures = data['Name']
ylabels = data[['Pattern Type','Neckline']]

In [6]:
# Split Data
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.1,random_state=30)

In [7]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),
                          ('lr_multi',MultiOutputClassifier(LogisticRegression()))])

In [8]:
# Fit on Dataset
pipe_lr.fit(x_train,y_train)

c:\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr_multi',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [9]:
# Accuracy Score
pipe_lr.score(x_test,y_test)

0.7494780793319415

In [15]:
text = 'striped shirt'

#print(pipe_RF.predict([text]))
print(pipe_lr.predict([text]))

[['Striped' 'Collar']]


In [11]:
import pickle

filename = 'finalized_pattern.sav'
pickle.dump(pipe_lr, open(filename, 'wb'))

loaded_pattern = pickle.load(open(filename, 'rb'))
result = loaded_pattern.score(x_test, y_test)
print(result)

0.7494780793319415


In [12]:
%store loaded_pattern

Stored 'loaded_pattern' (Pipeline)


In [13]:
%store -r loaded_model
%store -r good_sentences
%store -r new_data
%store -r loaded_pattern

import pandas as pd
import numpy as np
from itertools import chain
import seaborn as sns
import matplotlib as plt

pattern_data = []
for i in range (0, len(good_sentences), 1):
    x = loaded_pattern.predict([good_sentences[i]])
    pattern_data.append(x)
pattern_data = list(chain.from_iterable(pattern_data))
pattern_data = pd.DataFrame(pattern_data)

In [14]:
pattern_data

,0,1
0,Plain,Round Neck
1,Plain,Round Neck
2,Plain,Round Neck
3,Plain,Round Neck
4,Plain,High Neck
...,...,...
11747,Plain,NA
11748,Plain,NA
11749,Plain,NA
11750,Plain,NA
